In [4]:
import duckdb
import pandas as pd
from sodapy import Socrata
import config

client = Socrata("data.ny.gov",
                  config.API_TOKEN,
                  username=config.NY_OPEN_DATA_USERNAME,
                  password=config.NY_OPEN_DATA_PASSWORD)

results = client.get("39hk-dx4f", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [2]:
%load_ext sql
conn = duckdb.connect("mta.db")
%sql conn --alias duckdb



In [7]:
duckdb.sql("CREATE TABLE stations AS SELECT * FROM results_df")

In [3]:
duckdb.sql("select distinct line from stations")

CatalogException: Catalog Error: Table with name stations does not exist!
Did you mean "pg_settings"?

In [5]:
conn.close()

NameError: name 'conn' is not defined